Face Detection using Cascade Classiier 

In [1]:
import cv2
import time


# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# To capture video from webcam.
cap = cv2.VideoCapture(0)

In [2]:
while True:
    # Read the frame
    _, img = cap.read()

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the faces
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    # Draw the rectangle around each face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
    # check after every 5 seconds
    time.sleep(5)
    # Display
    k = cv2.waitKey(30) & 0xff
    cv2.imshow('img', img)
    
    print("Processing image...")
    
    # Stop if escape key is pressed
    if k == 27:
        break
        
    if k == ord('s'):
        
        cv2.imwrite(filename='saved_img2.jpg', img=img)
        img_new = cv2.imread('saved_img.jpg', cv2.IMREAD_GRAYSCALE)
        img_new = cv2.imshow("Captured Image", img_new)
        print("Image Captured")
        cv2.waitKey(1650)
        cv2.destroyAllWindows()
        print("Image saved Successfully on Local Drive")
        break
        


Processing image...
Processing image...
Processing image...
Image Captured
Image saved Successfully on Local Drive


![Result](https://trail-project.s3.amazonaws.com/at_door.jpg)

In [3]:
# Release the VideoCapture object
cap.release()

### AWS Command Line Interface

In [4]:
import logging
import boto3
from botocore.exceptions import ClientError
import os

Uploding Image to S3 Bucket

In [5]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [6]:
client = boto3.client('s3', region_name='us-east-1')
#image uploading
client.upload_file('saved_img.jpg', 'trail-project', 'at_door2.jpg')

print('Captured image is successfully uploaded on s3 bucket')

Captured image is successfully uploaded on s3 bucket


In [7]:
#Facial Comparison
import boto3
from pprint import pprint

client = boto3.client('rekognition')

Comparison Between Source Image and Target Image

![Input](https://cvproject99-s3.s3.amazonaws.com/hello.JPG)

In [8]:
import boto3

#Selecting captured Image and Comparing with Targeted Image
def face_comparision(value):
    client = boto3.client('rekognition','us-east-1')   #us-east-1 will be the region of our database bucket
    response = client.compare_faces(
        SourceImage={
            'S3Object': {
                'Bucket': 'trail-project',                #datacompare will be name of database of s3 bucket
                'Name': 'at_door.jpg'
            }
        },
        TargetImage={
            'S3Object': {
                'Bucket': 'cvproject99-s3',
                'Name': 'default.jpg'
            }
        },SimilarityThreshold=value

    )
    return (response)
response = face_comparision(90)
for record in face_comparision(0)['FaceMatches']:
    face = record
    confidence = face['Face']

    print("Matched With {:.5f}""%"" Similarity".format(face['Similarity']))
    print("With {:.5f}""%"" Confidence".format(confidence['Confidence']))

    c = float(format(face['Similarity']))


    if(c > 95):{
            print("Matched")
        }
    else:
        print("Not a match")
match = len(response['FaceMatches'])
total_people =  len(response['UnmatchedFaces'])+len(response['FaceMatches'])
if (len(response['UnmatchedFaces'])>1):
    print(total_people,' Out of',len(response['FaceMatches']),'Persons got matched')

Matched With 99.99792% Similarity
With 99.99921% Confidence
Matched


![Result](https://cvproject99-s3.s3.amazonaws.com/result.JPG)

In [9]:
#Facial Analysis
import boto3
from pprint import pprint

client = boto3.client('rekognition')

In [10]:
#Facial Analysis and Details 
# Import libraries
import json

# Specify bucket name and image key name
image_name = 'default.jpg'
bucket_name = 'cvproject99-s3'

# Invoke rekognition face
response = client.detect_faces(Image={'S3Object':{'Bucket':bucket_name, 'Name':image_name}}, Attributes=['ALL'])


for record in response['FaceDetails']:
    data = record
    confidence = face['Face']

    print("Gender :",record.get('Gender').get('Value'),", appears to be {:.2f}""%"" ".format(record.get('Gender').get('Confidence')))
    
    print("Age Range :",record.get('AgeRange').get('Low'),'-',record.get('AgeRange').get('High'),'years old')
    
    print('Smilling : {:.2f}%'.format(record.get('Smile').get('Confidence')))
    
    print('Eyeglasses :',record.get('Eyeglasses').get('Value'))
    
    print('MouthOpen :',record.get('MouthOpen').get('Value'))
    
    print('EyesOpen :',record.get('EyesOpen').get('Value'))


Gender : Male , appears to be 99.93% 
Age Range : 13 - 25 years old
Smilling : 99.44%
Eyeglasses : False
MouthOpen : False
EyesOpen : True


![Analysis](https://cvproject99-s3.s3.amazonaws.com/analysis.JPG)

In [11]:
#Analysis Done on Celebrity Image
import json

# Specify bucket name and image key name
image_name = 'Selena.jpg'
bucket_name = 'trail-project'

# Invoke rekognition face
response = client.detect_faces(Image={'S3Object':{'Bucket':bucket_name, 'Name':image_name}}, Attributes=['ALL'])


for record in response['FaceDetails']:
    data = record
    confidence = face['Face']

    print("Gender :",record.get('Gender').get('Value'),", appears to be {:.2f}""%"" ".format(record.get('Gender').get('Confidence')))
    
    print("Age Range :",record.get('AgeRange').get('Low'),'-',record.get('AgeRange').get('High'),'years old')
    
    print('Smilling : {:.2f}%'.format(record.get('Smile').get('Confidence')))
    
    print('Eyeglasses :',record.get('Eyeglasses').get('Value'))
    
    print('MouthOpen :',record.get('MouthOpen').get('Value'))
    
    print('EyesOpen :',record.get('EyesOpen').get('Value'))

Gender : Female , appears to be 99.70% 
Age Range : 23 - 35 years old
Smilling : 84.80%
Eyeglasses : False
MouthOpen : False
EyesOpen : False


![input](https://trail-project.s3.amazonaws.com/Selena.jpg)

In [12]:
#Comparison Between Peoples of group and Target Images 
import boto3
from pprint import pprint

client = boto3.client('rekognition')

In [13]:
import boto3


def face_comparision(value):
    client = boto3.client('rekognition','us-east-1')   #us-east-1 will be the region of your database bucket
    response = client.compare_faces(
        SourceImage={
            'S3Object': {
                'Bucket': 'trail-project',                #datacompare will be name of database of s3 bucket
                'Name': 'Rdj.jpg'
            }
        }, 
        TargetImage={
            'S3Object': {
                'Bucket': 'trail-project',
                'Name': '4_team.jpg'                     #Group of people as a traget Image
            }
        },SimilarityThreshold=value

    )
    return (response)

Comapring Images between Group of people and Target Source 

![Input](https://cvproject99-s3.s3.amazonaws.com/assemble.JPG)

In [14]:
response = face_comparision(90)
for record in face_comparision(0)['FaceMatches']:
    face = record
    confidence = face['Face']

    print("Matched With {:.5f}""%"" Similarity".format(face['Similarity']))
    print("With {:.5f}""%"" Confidence".format(confidence['Confidence']))

    c = float(format(face['Similarity']))


    if(c > 95):{
            print("Matched")
        }
    else:
        print("Not a match")
match = len(response['FaceMatches'])
total_people =  len(response['UnmatchedFaces'])+len(response['FaceMatches'])
if (len(response['UnmatchedFaces'])>1):
    print()
    print(total_people,' Out of',len(response['FaceMatches']),'Persons got matched')

Matched With 99.99975% Similarity
With 99.99659% Confidence
Matched
Matched With 0.94162% Similarity
With 99.99913% Confidence
Not a match
Matched With 0.92439% Similarity
With 99.99341% Confidence
Not a match
Matched With 0.12956% Similarity
With 99.99758% Confidence
Not a match

4  Out of 1 Persons got matched


![Results](https://cvproject99-s3.s3.amazonaws.com/simi.JPG)

In [15]:
#Msg format for Email and SMS Notification
msg = "Are you expecting guests?\n\nyou have " + str(total_people)+" visitors on the door.\n"+ str(total_people)+" Out of "+str(match)+" Persons got matched\n\n"+ "https://trail-project.s3.amazonaws.com/4_team.jpg"

In [16]:
print(msg)

Are you expecting guests?

you have 4 visitors on the door.
4 Out of 1 Persons got matched

https://trail-project.s3.amazonaws.com/4_team.jpg


Sending Outputs and Captured Image to our client 

In [17]:
#Initiating SNS Notification service 
client_sns = boto3.client('sns')

In [ ]:
#Sending Email and SMS to Client 
message = client_sns.publish(TargetArn='arn:aws:sns:us-east-1:923202930002:trail-project',
                         Message=msg,
                         Subject='Someone at the Door')